# Load Data

In [1]:
from proj1_helpers import *
from implementations import *
from cross_validation import *
import numpy as np

%load_ext autoreload
%autoreload 2

In [17]:
y, tx, ids = load_csv_data('Data/train.csv', False)
test_y, test_tx, test_ids= load_csv_data('Data/test.csv', False)

# Preprocessing

In [21]:
#print(test_y.shape)

PRI_jet_num_colomn_train = tx[:, 22]
PRI_jet_num_colomn_test = test_tx[:, 22]


y_clustered = list()
tx_clustered = list()
ids_clustered = list()
test_y_clustered = list()
test_tx_clustered = list()
test_ids_clustered = list()

for i in range(4):
    indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_train) if a == i]
    y_clustered.append(y[indices])
    tx_clustered.append(tx[indices])
    ids_clustered.append(ids[indices])
    
    test_indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_test) if a == i]
    test_y_clustered.append(test_y[test_indices])
    test_tx_clustered.append(test_tx[test_indices])
    test_ids_clustered.append(test_ids[test_indices])
    
    #delete colinear columns
    indices_to_delete = list()
    for col in range(tx_clustered[i].shape[1]):
        if min(tx_clustered[i][:, col]) == max(tx_clustered[i][:, col]):
            indices_to_delete.append(col)

    tx_clustered[i] = np.delete(tx_clustered[i], indices_to_delete, 1)
    test_tx_clustered[i] = np.delete(test_tx_clustered[i], indices_to_delete, 1)

# Cross-Validation & Weight computation

In [22]:
from cross_validation import *
w = list()
result = list()

for i in range(4):
    w.append(cross_validation_run(tx_clustered[i], y_clustered[i]))
    test_y_clustered[i] = predict_labels(w[i], test_tx_clustered[i])

Final: loss train:  0.391871113688  loss test:  0.392181236779  weights:  [ -2.72718369e-02  -2.69456774e-01  -4.23726697e-01   4.03466332e+02
   3.76454228e-01  -4.03507279e+02  -6.09365988e+00  -1.73838876e-01
   3.08024060e-02   4.14032255e+00  -1.07130893e-03  -6.39111356e-04
   4.00758533e+00   6.64493415e-03   5.91913410e-05   9.28931646e-02
  -5.37436259e-03   3.51637468e-02]   (18,)
Final: loss train:  0.41303649184  loss test:  0.413460841985  weights:  [  3.19982757e-02  -2.19860696e-01  -2.48927144e-01  -8.80748536e-02
   2.40295994e-01   5.37928247e-02  -2.54856373e+02  -1.69125547e-01
   1.40712452e-01   8.23260910e+01   4.12771032e-03  -3.45116418e-03
   8.36786056e+01  -7.82637756e-03   9.90590151e-04   8.76595854e-02
   4.12270525e-03  -4.15677106e-02  -3.20824460e+03  -7.98267366e-04
  -2.10968909e-03   3.38488503e+03]   (22,)
Final: loss train:  0.354906884399  loss test:  0.355614393838  weights:  [  4.45213839e-02  -1.55395381e-01  -1.72486704e-01   1.18383831e-01
 

# Post Processing

In [23]:
test_ids = [item for sublist in test_ids_clustered for item in sublist]
y_pred = [item for sublist in test_y_clustered for item in sublist]

test_ids, y_pred = zip(*sorted(zip(test_ids, y_pred)))

# Output Data

In [24]:
name = 'second-submission'
create_csv_submission(test_ids, y_pred, name)